In [1]:
import requests
import json
import pandas as pd

In [6]:
with open('keyfile') as keyfile:
    API_KEY= keyfile.read()
    
## Geography
london_box = (51.145756,-0.913796,51.955268,0.388083)
london_mid = (51.5,-0.12)
# Trafalgar Square
uk_box = (49.3739906, -7.0113373, 61.2291139, 2.4798203)
# Isles of Scilly to Shetland Islands (includes a bit of belgium and most of ROI)

# Requests
url = 'https://airlabs.co/api/v9/'
endpoint = lambda e: url + e

# Get Static Data
with open('airports.json') as airport_data:
    airports = json.load(airport_data)
    
with open('ldn_airports.json') as ldn_airports_file:
    ldn_airports = json.load(ldn_airports_file)
    
ldn_names = [a['name'] for a in ldn_airports]    

# Dictionary for turning iata_codes into airport names
#TO DO: add cities to this

codes = {ap['iata_code']:ap['name'] 
         for ap in airports if 'iata_code' in ap.keys()
        }

In [53]:
# Request live data

r= requests.get(endpoint('flights'),
                params={
                    'api_key':API_KEY,
                    'bbox':london_box,
                }
               )
all_planes = r.json()['response']
flying_planes = [plane for plane in all_planes if plane['status'] == 'en-route']

# make DataFrame

df= pd.DataFrame(flying_planes)
#print(df.columns); df.head()

# Make the dataframe more user friendly

df['Source'] = df['dep_iata'].map(lambda x : codes[x], 
                                  na_action='ignore')
df['Destination'] = df['arr_iata'].map(lambda x : codes[x], 
                                       na_action='ignore')

# Divide frames into arrivals, departures, fly-overs, mystery and intra-london (naughty) flights
df_to_london = df.query("Destination in @ldn_names")
df_from_london = df.query("Source in @ldn_names")
df_flyover = df.query(
    "Source not in @ldn_names and Destination not in @ldn_names").dropna()
df_mystery = df.query('Destination.isna()')
df_naughty = df_to_london.query('Source is in @ldn_names')

# cheeky Lambda for representing the dataframes
show_frame = lambda df : df[['Source','Destination','alt','reg_number','flag']].sort_values(['alt','Destination'])

,hex,reg_number,flag,lat,lng,alt,dir,speed,v_speed,squawk,...,dep_iata,arr_icao,arr_iata,airline_icao,airline_iata,aircraft_icao,updated,status,Source,Destination
10,402988,G-ECRM,GB,51.334805,-1.299637,914,78,170,0.0,7004,...,NaN,NaN,NaN,NaN,NaN,RF6,1674750561,en-route,NaN,NaN
11,403017,G-KKRN,GB,50.825100,-0.228035,-60,296,74,-2.2,3763,...,NaN,NaN,NaN,NaN,NaN,R22,1674750560,en-route,NaN,NaN
12,403723,G-BUUC,GB,51.506424,-1.470089,883,228,231,2.0,7004,...,NaN,NaN,NaN,NaN,NaN,RF6,1674750562,en-route,NaN,NaN
13,404598,G-BZXZ,GB,51.785873,-0.928192,396,341,162,1.1,7000,...,NaN,NaN,NaN,NaN,NaN,BDOG,1674750561,en-route,NaN,NaN
16,406124,G-SPTR,GB,51.429230,-1.073600,243,9,116,0.9,0036,...,NaN,NaN,NaN,NaN,NaN,R44,1674750563,en-route,NaN,NaN
17,407063,G-CODA,GB,50.955872,-0.634460,259,292,209,0.6,3763,...,NaN,NaN,NaN,NaN,NaN,H500,1674750562,en-route,NaN,NaN
18,407160,G-RJVH,GB,51.671001,-0.334000,0,282,105,NaN,7000,...,NaN,NaN,NaN,NaN,NaN,G2CA,1674750549,en-route,NaN,NaN
19,407193,G-COGS,GB,51.168869,0.087509,190,40,185,0.0,3750,...,NaN,NaN,NaN,NaN,NaN,B407,1674750563,en-route,NaN,NaN
31,400D0E,G-OHLI,GB,51.443665,-1.056766,236,38,138,0.5,7000,...,NaN,NaN,NaN,NaN,NaN,R44,1674750562,en-route,NaN,NaN
32,400E16,G-CTCB,US,51.550552,0.648117,76,58,166,-2.8,1250,...,NaN,NaN,NaN,LFA,NaN,DA42,1674750562,en-route,NaN,NaN


### If there are any planes flying between london airports AND they've put this in their trackers, they will show up here:

In [13]:
if len(df_naughty)>0:
    show_frame(df_naughty)
else:
    print("no planes")

NameError: name 'df_naughty' is not defined

In [51]:
df_flyover = df.query("Source not in @ldn_airport_names and Destination not in @ldn_airport_names").dropna()

In [52]:
show_frame(df_flyover)

,Source,Destination,alt,reg_number,flag
7,Paris Charles de Gaulle Airport,Minneapolis-St Paul International Airport,10363,N826NW,US
6,"Oslo Airport, Gardermoen",Gran Canaria Airport,10668,LN-ENN,NO
5,Poznan-Lawica Airport,Birmingham International Airport,10721,SP-RST,IE
4,Dublin Airport,Brussels Airport,11277,EI-CVB,IE
3,Cologne Bonn Airport,Dublin Airport,11582,EI-DHS,IE
0,Agadir-Al Massira Airport,Manchester Airport,11582,G-RUKC,UK
1,Alicante-Elche Airport,Manchester Airport,11582,G-JZHV,UK
2,Munich International Airport,Montreal-Pierre Elliott Trudeau International ...,11582,D-AIVD,DE


In [ ]:
show_frame(df_from_london)

In [ ]:
show_frame(df_to_london)